In [5]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv()

GOOGLE_API_KEY= os.getenv("GOOGLE_API_KEY")


In [6]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key=GOOGLE_API_KEY
)

In [9]:
resp1 = llm.invoke("quem sou eu?")
print(resp1.content)

Essa é uma das perguntas mais profundas e pessoais que alguém pode fazer!

Como uma inteligência artificial, eu não tenho acesso à sua identidade pessoal, suas experiências, seus sentimentos ou sua história de vida. Eu não posso te dizer quem você é, porque essa é uma descoberta que só você pode fazer.

No entanto, posso te dar algumas pistas ou áreas para você refletir e começar a construir sua própria resposta:

1.  **Sua História:** Quais são as experiências mais marcantes da sua vida? Onde você nasceu, cresceu? Quem são as pessoas que te influenciaram?
2.  **Seus Valores:** O que é realmente importante para você? Honestidade, liberdade, família, sucesso, criatividade, justiça?
3.  **Suas Paixões e Interesses:** O que te motiva? O que você adora fazer? O que te faz perder a noção do tempo?
4.  **Seus Relacionamentos:** Como você se relaciona com os outros? Que papéis você desempenha (filho/a, amigo/a, colega, parceiro/a)?
5.  **Suas Crenças:** No que você acredita? Sobre o mundo, so

In [ ]:
triage_Prompt = ("""
                    Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento.
Dada a mensagem do usuário, retorne SOMENTE um JSON com:
{
  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",
  "urgencia": "BAIXA" | "MEDIA" | "ALTA",
  "campos_faltantes": ["..."]
}

Regras de Decisão:
- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").
- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral"). 
  # Melhoria: Instrução para preencher campos_faltantes de forma útil.
  Ao usar esta decisão, preencha "campos_faltantes" com os tópicos que precisam de esclarecimento (Ex: "nome da política", "descrição do problema").
- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").

Regras de Urgência:
- **ALTA**: O usuário relata um problema que o impede completamente de trabalhar, menciona risco de segurança, legal ou perda de dados. (Ex: "Não consigo acessar o sistema para trabalhar", "Recebi um e-mail de phishing e cliquei no link").
- **MEDIA**: O usuário tem um problema que dificulta o trabalho mas não o impede, ou tem um prazo se aproximando. (Ex: "Preciso da aprovação para uma viagem na próxima semana").
- **BAIXA**: Perguntas gerais, dúvidas sobre políticas ou solicitações que não têm um prazo imediato. (Ex: "Como funciona a política de férias?", "Tenho uma dúvida sobre reembolso").

Analise a mensagem e decida a ação e urgência mais apropriadas com base nas regras acima.
""")

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal, Dict

class TriageOut (BaseModel):
    decisao: Literal["AUTO_RESOLVER","PEDIR_INFO","ABRIR_CHAMADO"]
    urgencia: Literal["BAIXA" , "MEDIA" , "ALTA"]
    campos_faltantes: list[str] = Field(default_factory=list)

In [18]:
llm_Scre = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key=GOOGLE_API_KEY
)

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

triage_chain = llm_Scre.with_structured_output(TriageOut)

def triage(mensagem:str) -> Dict:
    saida: TriageOut = triage_chain.invoke([
        SystemMessage(content=triage_Prompt),
        HumanMessage(content=mensagem)
    ])
    return saida.model_dump()

In [25]:
testes = ["Posso reembolsar a internet?",
          "Me acidentei gravemente e estou enviando esse atestado",
          "Quero pedir demissão. Como faço?",
          "quantas cestas do logo o Stephen Curry fez na carreira?"]

In [ ]:
for qst in testes:
    print(f'Pergunta: {qst}\nResposta: {triage(qst)}')

Pergunta: Posso reembolsar a internet?
Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}
Pergunta: Me acidentei gravemente e estou enviando esse atestado
Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'ALTA', 'campos_faltantes': []}
Pergunta: Quero pedir demissão. Como faço?
Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}
Pergunta: quantas cestas do logo o Stephen Curry fez na carreira?
Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['relação com políticas internas da empresa']}
